In [ ]:
import os
os.chdir("/home/znai/tgbot/tgbotsb/cb_ranker")

In [ ]:
import json
import pyarrow
from omegaconf import OmegaConf
from src.cb_ranker import CatBoostRank
from catboost import CatBoostRanker, Pool
import pandas as pd
import numpy as np
import polars as pl
import matplotlib.pyplot as plt
config = OmegaConf.load("configs/cb_ranker_config.yaml")


# iterations: 1000
# learning_rate: 0.03
# depth: 6
# verbose: 100



In [ ]:
train_df = pd.read_parquet('dataset/train_df.pq', engine='pyarrow')
val_df = pd.read_parquet('dataset/val_df.pq', engine='pyarrow')

In [ ]:
grouped_val_sum = val_df.groupby(['query_rank']).sum()
grouped_val_count = val_df.groupby(['query_rank']).count()
grouped_train_sum = train_df.groupby(['query_rank']).sum()
grouped_train_count = train_df.groupby(['query_rank']).count()
zero_val_ranks = grouped_val_sum[grouped_val_sum['feedback'] == 0].index.tolist()
no_full_group_val = grouped_val_sum[grouped_val_sum['feedback'] == 1].index.tolist()
zero_train_ranks = grouped_train_sum[grouped_train_sum['feedback'] == 0].index.tolist()
no_full_group_train = grouped_train_sum[grouped_train_sum['feedback'] == 1].index.tolist()
drop_group_val = zero_val_ranks + no_full_group_val
drop_group_train = zero_train_ranks + no_full_group_train
train_df = train_df[~train_df['query_rank'].isin(drop_group_train)]
val_df = val_df[~val_df['query_rank'].isin(drop_group_val)]

In [ ]:

ranker = CatBoostRank(config=config,
        cat_features=['is_release',"is_tech_light"],
        group_id_column='query_rank',
        target_column='feedback'  # Или 'MVS'
)

ranker.fit(train_df)
predictions = ranker.predict(val_df)

val_metrics = ranker.evaluate(val_df)

In [ ]:
val_metrics

In [ ]:
val_df[:10]

In [44]:

ranker.model.save_model("cbranker")
